In [121]:
# Import Libraries
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules
import matplotlib.pyplot as plt
from pyECLAT import ECLAT

In [122]:
# Read the dataset 
df = pd.read_csv('retail_dataset.csv', sep=',')

# Print the first 5 rows
df.head(5)

,0,1,2,3,4,5,6
0,Bread,Wine,Eggs,Meat,Cheese,Pencil,Diaper
1,Bread,Cheese,Meat,Diaper,Wine,Milk,Pencil
2,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
3,Cheese,Meat,Eggs,Milk,Wine,NaN,NaN
4,Meat,Pencil,Wine,NaN,NaN,NaN,NaN


In [123]:
# Find the unique items in the table
items = set()
for col in df:
    items.update(df[col].unique())
print(items)

{'Meat', 'Cheese', 'Bread', 'Wine', nan, 'Pencil', 'Bagel', 'Eggs', 'Diaper', 'Milk'}


In [124]:
itemset = set(items)
encoded_vals = []
for index, row in df.iterrows():
    rowset = set(row) 
    labels = {}
    uncommons = list(itemset - rowset)
    commons = list(itemset.intersection(rowset))
    for uc in uncommons:
        labels[uc] = 0
    for com in commons:
        labels[com] = 1
    encoded_vals.append(labels)
encoded_vals[0]
ohe_df = pd.DataFrame(encoded_vals)

fpgrowth

In [125]:
%%time
# Applying fpgrowth
freq_items = fpgrowth(ohe_df, min_support=0.1, use_colnames=True)

# Mining association rules
fpgrowth_rules = association_rules(freq_items, metric="confidence", min_threshold=0.5)

CPU times: total: 0 ns
Wall time: 8 ms


c:\Users\paizc\AppData\Local\Programs\Python\Python311\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [132]:
# Finding the support of the itemset
support_milk_meat_cheese = freq_items[freq_items['itemsets'] == frozenset({'Milk', 'Meat', 'Cheese'})]['support'].values[0]

print("Rate of Milk, Meat, and Cheese being purchased together:", support_milk_meat_cheese)

Rate of Milk, Meat, and Cheese being purchased together: 0.20317460317460317
Percentage of customers who buy Eggs, Meat, and Cheese: 21.58730158730159


In [127]:
# Finding the support of the itemset
support_milk_meat_cheese = freq_items[freq_items['itemsets'] == frozenset({'Eggs', 'Meat', 'Cheese'})]['support'].values[0]

print("Percentage of customers who buy Eggs, Meat, and Cheese:", support_milk_meat_cheese*100, "%")

Percentage of customers who buy Eggs, Meat, and Cheese: 21.58730158730159 %


ECLAT

In [128]:
attrCount = len(df.count())
i = 0
for col in df.columns[:attrCount].tolist():
    df.rename(columns={col : i}, inplace=True)
    i += 1

In [129]:
%%time

# Init ECLAT
eclat = ECLAT(df)

# Apply ECLAT
x, y = eclat.fit(min_support=0.1)

Combination 1 by 1


9it [00:00, 449.90it/s]


Combination 2 by 2


36it [00:00, 473.58it/s]


Combination 3 by 3


84it [00:00, 515.22it/s]

CPU times: total: 281 ms
Wall time: 280 ms


In [130]:
for key, value in y.items():
    if key == 'Meat & Cheese & Milk':
        support = value
        break

print("Rate of Milk, Meat, and Cheese being purchased together:", support)

Rate of Milk, Meat, and Cheese being purchased together: 0.20317460317460317
